<a href="https://colab.research.google.com/github/Riyakumarint/Authentication/blob/main/MAJOR_PROJECT%20%3A%20Automatic_fault_detection_using_neural_network_in_PV_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from sklearn.datasets import load_iris
from sklearn import tree
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [103]:
classData = pd.read_csv('Self_Fault Created Data In Matlab.csv')

In [104]:
classData.shape

(25, 7)

In [106]:
classData.head()

,irr,pvt,i1,v1,i2,v2,f
0,1000,25,7.7025,77.833,7.7025,77.833,0
1,1000,25,3.8057,92.142,3.8057,92.142,0
2,1000,25,2.6002,94.412,2.6002,94.412,0
3,1000,25,6.1210,88.513,6.1210,88.513,0
4,1000,25,1.8986,95.636,1.8986,95.636,0


In [107]:
Xclass = classData.iloc[:, 0:6].values
yclass = classData.iloc[:, 6:].values

In [108]:
print(Xclass[0,:])

[1000.       25.        7.7025   77.833     7.7025   77.833 ]


In [109]:
print(yclass[0,:])

[0]


In [110]:
faults = ["None", "Short Circuit Fault", "Degradation", "Open Circuit Fault", "Partial Shadow"]
yc = []
for f in yclass:
  if f[0] == 0:
    yc.append([1., 0., 0., 0., 0.])
  elif f[0] == 1 :
    yc.append([0., 1., 0., 0., 0.])
  elif f[0] == 2 :
    yc.append([0., 0., 1., 0., 0.])
  elif f[0] == 3 :
    yc.append([0., 0., 0., 1., 0.])
  elif f[0] == 4 :
    yc.append([0., 0., 0., 0., 1.])

yc = np.array(yc)
yclass = yc

In [111]:
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xclass, yclass, test_size = 0.2, random_state = 0)

In [112]:
sc = StandardScaler()
Xc_train = sc.fit_transform(Xc_train)
# print(Xc_train)
Xc_test = sc.transform(Xc_test)
# print(Xc_test)

### ANN Model

In [117]:
# Build neural network
classANN = Sequential()
classANN.add(Dense(units=12, activation='relu'))
classANN.add(Dense(units=12, activation='relu'))
classANN.add(Dense(units=5, activation='sigmoid'))

# Compile model
classANN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [121]:
# Train model
classANN.fit(Xc_train, yc_train, batch_size = 32, epochs = 100)

Epoch 1/100
1/1 [==============================] - 0s 13ms/step - loss: 1.1402 - accuracy: 0.6500
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 1.1360 - accuracy: 0.6500
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 1.1319 - accuracy: 0.6500
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 1.1278 - accuracy: 0.6500
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 1.1237 - accuracy: 0.6500
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 1.1197 - accuracy: 0.6500
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 1.1157 - accuracy: 0.6500
Epoch 8/100
1/1 [==============================] - 0s 7ms/step - loss: 1.1117 - accuracy: 0.6500
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 1.1078 - accuracy: 0.6500
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 1.1039 - accuracy: 0.6500
Epoch 11/100
1/1 [=======

In [122]:
classANN.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 12)                84        
                                                                 
 dense_41 (Dense)            (None, 12)                156       
                                                                 
 dense_42 (Dense)            (None, 5)                 65        
                                                                 
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________


In [123]:
loss, acc = classANN.evaluate(Xc_test, yc_test)
print(f'Accuracy : {acc*100} %')

1/1 [==============================] - 0s 18ms/step - loss: 0.9972 - accuracy: 0.8000
Accuracy : 80.0000011920929 %


In [124]:
# test_xc = sc.transform([[1.007, 2.42, 0.06, 0.73, 0.006, 0.57 ]]) # no fault
# test_xc = sc.transform([[989.377, 43.95, 9.26, 258.987, 9.11, 201.5 ]]) # short circuit fault
# test_xc = sc.transform([[961.167, 42.39, 9.07, 265.58, 8.5, 247.01 ]]) # degradation
# test_xc = sc.transform([[990.15, 44.9, 0.1, 1.6, 9.1, 267.1 ]]) # open circuit fault
#test_xc = sc.transform([[833.3, 40.5, 7.89, 269.7, 4.17, 316.9 ]]) # partial shadow
#test_xc = sc.transform([[990,25,1.45,60,4,80]])
test_xc = sc.transform([[990,25,3.15,68,8,70]])


result_xc = classANN.predict(test_xc)
n = 0
temp=0.0
for i in range(5) :
  if(result_xc[0, i] >temp) :
    temp = result_xc[0, i]
    n=i

print(faults[n])

1/1 [==============================] - 0s 51ms/step
Open Circuit Fault
